In [ ]:
# now open in napari
import napari
import tifffile
import os
import numpy as np
import matplotlib.pyplot as plt
import yaml

from photostim_deve.control_exp.io import get_med_img_s2p

from photostim_deve.response.io import parse_mark_points, mp_dict_to_stim_list, load_photostim_protocol, get_all_tiff_paths
from photostim_deve.response.compute import get_fov_resp, get_fov_resp_mn_md, get_dist_dff, compute_dist_kernel
from photostim_deve.response.plot import plot_xyoff, plot_protocol, plot_fov_diff_single, plot_fov_all_point, zscore_act, plot_dist_dff, plot_fov_map, plot_kernel_2d, plot_fov_map_avg, plot_raster_matched_rois, plot_raster_matched_rois_avg, plot_response_matched_rois, plot_response_matched_rois_heatmap, plot_response_matched_rois_avg

%load_ext autoreload
%autoreload 2


In [ ]:
# 1) Set parameters
data_dir = 'data_proc' # data_loc is  the directory on local ssd (only two sessions, one for jm049 and one for jm048)
experimenter = 'jm'
mouse = 'jm064' # 'jm049' or 'jm048'
session =  '2025-11-13_calib' # '2025-05-23_b' or '2025-05-08_c'
protocol = 'power' # 'power', 'size', 'xshift' ('yshift'), 'zshift'

coord_invert = False # only applies to jm048 sessions (didnt do calib for that mouse)

In [ ]:
# load paramters from config file

with open("resp_map_config.yaml", "r") as f:
    cfg = yaml.safe_load(f)

channel = cfg["channel"]
plane = cfg["plane"]
frame_period = cfg["frame_period"]
fov_shape = cfg["fov_shape"]

bsln_n_frames = cfg["bsln_n_frames"]
resp_n_frames = cfg["resp_n_frames"]

bsln_sub_type = cfg["bsln_sub_type"]

n_dist_bins = cfg["n_dist_bins"]

n_rows_fov = cfg["n_rows_fov"]
vlim = cfg["vlim"]
txt_shift = cfg["txt_shift"]
sat_perc_fov = cfg["sat_perc_fov"]
peristim_wind = cfg["peristim_wind"]
zoomin_npix = cfg["zoomin_npix"]

dist_bins_xlim = cfg["dist_bins_xlim"]
dist_bins_xlim_zoom = cfg["dist_bins_xlim_zoom"]

In [ ]:
# now get all the suite2p paths
data_path = os.path.join(data_dir, experimenter, mouse, session, protocol)

# now find all the subfolders and print them as conditions
all_x_val_path = [f.path for f in os.scandir(data_path) if f.is_dir()]
all_x_val_path.sort()
all_x_val_str = [os.path.basename(f) for f in all_x_val_path]
# all_x_val = np.array([float(f.split('ms')[0]) for f in all_x_val_str])
# do similar as above, but just take the part that are numbers (in case there are other characters)
all_x_val = []
for f in all_x_val_str:
    num_str = ''.join([c for c in f if c.isdigit() or c == '.' or c == '-'])
    try:
        all_x_val.append(float(num_str))
    except:
        all_x_val.append(f)  # if conversion fails, keep the original string
all_x_val = np.array(all_x_val)

# now re-sort both based on the value of all_x_val
sort_idxs = np.argsort(all_x_val)

all_x_val = all_x_val[sort_idxs]
all_x_val_str = [all_x_val_str[idx] for idx in sort_idxs]
all_x_val_path = [all_x_val_path[idx] for idx in sort_idxs]

print("Found the following conditions:")
for i, f in enumerate(all_x_val_str):
    print(f"{i}: {f} (x_val: {all_x_val[i]})")

In [ ]:
for (i, session_path) in enumerate(all_x_val_path):  # just
    print(session_path)

    # IMPORTANT: Code below is from resp_map.ipynb
    # tiff file paths
    s2p_path = os.path.join(session_path, 'suite2p', f'plane{plane}')
    tiff_dir = os.path.join(s2p_path, f'reg_tif_chan{channel}')
    all_tiff_paths = get_all_tiff_paths(tiff_dir)

    # stimulation protocol pathsa
    csv_save_path = os.path.join(data_dir, experimenter, mouse, session, 'photostim_protocol.csv')
    csv_load_path = csv_save_path

    # output paths
    output_path = os.path.join(session_path, 'photostim_deve')
    output_fig_path = os.path.join(output_path, 'fig')

    if not os.path.exists(output_path):
        os.makedirs(output_path)
    if not os.path.exists(output_fig_path):
        os.makedirs(output_fig_path)

    # loading suite2p data
    meds, mn_image, s2p_idxs, ops, f = get_med_img_s2p(session_path)
    xoff = ops['xoff']
    yoff = ops['yoff']

    plot_xyoff(xoff, yoff, save_path=os.path.join(output_fig_path, 'xyoff.png'))

    # Load stim protocol
    mp_dict = parse_mark_points(session_path)
    for key, value in mp_dict.items():
        print(f"Key: {key}, Value: {value}")

    _ = mp_dict_to_stim_list(mp_dict, frame_period=frame_period, fov_shape=fov_shape, csv_save_path=csv_save_path)

    all_time, all_frame, all_point, all_coords_x, all_coords_y = load_photostim_protocol(csv_load_path)

    if coord_invert: # invert coordinates if needed (jm048)
        all_coords_x, all_coords_y = fov_shape[0] - all_coords_x, fov_shape[1] - all_coords_y

    n_points = len(np.unique(all_point))
    fov_bsln, fov_resp, fov_diff = get_fov_resp(all_tiff_paths, all_frame, bsln_n_frames=bsln_n_frames, resp_n_frames=resp_n_frames, fov_shape=fov_shape)

    if bsln_sub_type == 'trial_by_trial': # subtract the baseline in each trial of each point independently.
        fov_map, _ = get_fov_resp_mn_md(fov_diff, all_point)

    elif bsln_sub_type == 'session_wide': # subtract the baseline across all trials of all points (mean of all those). 
        fov_resp_mn = get_fov_resp_mn_md(fov_resp, all_point)
        fov_bsln_glob_mean = np.nanmean(fov_bsln, axis=0)
        fov_map = fov_resp_mn - fov_bsln_glob_mean

    plot_fov_all_point(mn_image, all_point, all_coords_x, all_coords_y, txt_shift=txt_shift, save_path=os.path.join(output_fig_path, 'fov_mn_markpoints.png'), sat_perc=sat_perc_fov)
    dist_diff_mn, dist_diff_std = get_dist_dff(fov_map, all_point, all_coords_x, all_coords_y, fov_shape=fov_shape, n_dist_bins=n_dist_bins)
    plot_dist_dff(dist_diff_mn, n_points=n_points, dist_bins_xlim=dist_bins_xlim, dist_bins_xlim_zoom=dist_bins_xlim_zoom, save_path=os.path.join(output_fig_path, 'dist_dff.png'))

    k1d, k2d = compute_dist_kernel(dist_diff_mn, n_dist_bins=n_dist_bins)
    plot_kernel_2d(k2d, fov_shape=fov_shape, n_dist_bins=n_dist_bins, vlim=vlim, save_path=os.path.join(output_fig_path, 'kernel_2d.png'))
    plot_kernel_2d(k2d, fov_shape=fov_shape, n_dist_bins=n_dist_bins, vlim=vlim, save_path=os.path.join(output_fig_path, 'kernel_2d_zoomin.png'), zoomin_npix=zoomin_npix)

    plot_fov_map(fov_map, all_coords_x, all_coords_y, vlim=vlim, save_path=os.path.join(output_fig_path, 'fov_map.png'), n_rows=n_rows_fov)
    plot_fov_map(fov_map, all_coords_x, all_coords_y, vlim=vlim, save_path=os.path.join(output_fig_path, 'fov_map_zoomin64.png'), n_rows=n_rows_fov, zoomin_npix=zoomin_npix)

    all_point_med_idx = np.zeros(n_points, dtype=int)  # to store the index of the med image closest to each stimulation point


    # compute some quick summary for calibration (get suite2p-independent response around the stimulation point)
    if '_calib' in session:
        resp_px_rad = 10 # radius in pixels to average response around the stimulation point
        resp_px = np.mean(dist_diff_mn[:,:resp_px_rad], axis=1)
        print("Average response in the pixels around the stimulation point:", resp_px)
        # save it in the 'time' directory as resp_px_##ms.npy
        np.save(os.path.join(data_path, f'resp_px_{all_x_val_str[i]}.npy'), resp_px)

